In [ ]:
import matplotlib.pyplot as plt

from monai.data import DataLoader, Dataset
from monai.utils import set_determinism, first
from monai.transforms import (
    LoadImaged,
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    ScaleIntensityRanged,
    Orientationd,
    Spacingd,
    ToTensord
)

from transforms import IoUCropd

In [ ]:
data = [
    {
        'img': '../data/FLARETs_0047_0000.nii.gz',
        'my_key': 1
    },
    {
        'img': '../data/FLARETs_0048_0000.nii.gz',
        'my_key': 2
    },
    {
        'img': '../data/FLARETs_0049_0000.nii.gz',
        'my_key': 2
    }
]

In [ ]:
# TODO: check invalid metadata for orientation
# https://github.com/Project-MONAI/MONAI/discussions/3906

In [ ]:
transforms = Compose([
    LoadImaged(keys=['img']),
    EnsureChannelFirstd(keys=['img']),
    Orientationd(keys=['img'], axcodes='RAS'),
    Spacingd(keys=['img'], pixdim=(1.5, 1.5, 2.0), mode=('bilinear')),
    ScaleIntensityRanged(keys=['img'], a_min=-175, a_max=250,
                         b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=['img'], source_key='img'),
    IoUCropd(keys=['img'], min_iou=0.6, max_iou=0.7, debug=True),
    # TODO: put augmentations here
    ToTensord(keys=['img1', 'img2'])
])

check_ds = Dataset(data=data, transform=transforms)
check_loader = DataLoader(check_ds, batch_size=1)

In [ ]:
check_data = first(check_loader)
img = check_data['img'][0][0]

In [ ]:
plt.imshow(check_data['img'][0][0][:, :, 48], cmap='gray', vmin=0, vmax=1)

In [ ]:
print(check_data['img1_coords'])
plt.imshow(check_data['img1'][0][0][:, :, 0], cmap='gray', vmin=0, vmax=1)

In [ ]:
print(check_data['img2_coords'])
plt.imshow(check_data['img2'][0][0][:, :, 11], cmap='gray', vmin=0, vmax=1)